In [ ]:
import zipfile
import os
import pdfplumber
import pandas as pd

In [ ]:
# Extrair arquivos do ZIP em uma pasta chamada "extracao"
zip_path = "Anexos.zip"
extract_path = "extracao"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Nome do arquivo específico que queremos processar
arquivo_alvo = "Anexo_I.pdf"
pdf_path = os.path.join(extract_path, arquivo_alvo)



In [ ]:
# Função para garantir que os nomes das colunas sejam únicos
def renomear_colunas(colunas):
    colunas_unicas = []
    contador = {}
    
    for col in colunas:
        if col not in colunas_unicas:
            colunas_unicas.append(col)
            contador[col] = 1
        else:
            novo_nome = f"{col}_{contador[col]}"
            while novo_nome in colunas_unicas:
                contador[col] += 1
                novo_nome = f"{col}_{contador[col]}"
            colunas_unicas.append(novo_nome)
            contador[col] += 1
    
    return colunas_unicas


In [ ]:
# Criar lista para armazenar as tabelas do Anexo_I.pdf
tabelas = []

# Processar apenas o arquivo Anexo_I.pdf
if os.path.exists(pdf_path):  
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table and len(table) > 1:  # Verifica se há dados além do cabeçalho
                    df = pd.DataFrame(table[1:], columns=table[0])

                    # Renomear colunas duplicadas
                    df.columns = renomear_colunas(df.columns)

                    tabelas.append(df)
    except Exception as e:
        print(f"Erro ao processar {arquivo_alvo}: {e}")

    # Se houver tabelas extraídas, salvar todas em um único CSV
    if tabelas:
        df_final = pd.concat(tabelas, ignore_index=True)  # Une todas as tabelas extraídas
        df_final.to_csv("tabela_anexo_1.csv", index=False)
        print("Todas as tabelas extraídas do Anexo_I foram salvas em tabela_anexo_1.csv")
    else:
        print("Nenhuma tabela foi extraída do Anexo_I.pdf.")
else:
    print(f"Arquivo {arquivo_alvo} não encontrado no ZIP extraído.")

In [ ]:
# Carregar o CSV gerado
df = pd.read_csv("tabela_anexo_1.csv")

# Renomear as colunas OD e AMB, se existirem no arquivo
df.rename(columns={"OD": "Seg. Odontológica", "AMB": "Seg. Ambulatorial"}, inplace=True)

# Salvar o arquivo atualizado mantendo o mesmo nome
df.to_csv("tabela_anexo_1.csv", index=False)

print("Colunas renomeadas e arquivo atualizado com sucesso!")